# Pushing and processing assembly results

In [11]:
phages=open("./data/K2.151228_PacBio_Nahant_Phages.txt").readlines()
phages=[i.replace("\r\n","") for i in phages]
groups=open("/Users/jmb/Desktop/ViralFate/OrthoMCL/tSNE/tSNE_genome_clusters.txt").readlines()
plist=[i.split("\t")[0] for i in groups[1:]]
grplist=[i.split("\t")[-1].replace("\n","") for i in groups[1:]]
pgd=dict(zip(plist, grplist))
pgd

for p in phages:
    print("%s\t%s" % (p, pgd[p]))

1.080.O.	5
1.043.O.	5
1.095.O.	5
1.107.C.	5
1.020.O.	5
1.008.O.	5
1.249.A.	0
1.190.O.	25
1.259.O.	24
2.159.B.	12
1.028.O.	12
1.257.O.	23
1.082.O.	17
1.176.O.	17
1.228.O.	17
1.148.O.	17
1.233.B.	17
1.189.O.	17
1.182.O.	17
1.074.O.	17
1.165.O.	17
1.154.O.	15
1.211.B.	27
1.110.O.	17
1.275.O.	28
1.131.O.	17
1.239.O.	17
1.111.B.	21
1.216.O.	22
12G01.O.	1
1.054.O.	9
1.090.B.	1
1.135.O.	22
1.236.O.	22
1.052.A.	15
1.209.O.	8
1.185.O.	10
1.291.O.	14
1.077.O.	18
1.204.O.	16
1.151.O.	22
1.276.O.	22
1.144.O.	22
1.009.O.	3
1.122.B.	7
1.231.O.	22
2.095.B.	22
2.096.O.	15
2.044.O.	2
1.177.O.	22
1.087.A.	3
1.030.O.	3
1.123.O.	4
1.029.O.	9
1.253.O.	32
1.262.O.	30
1.210.O.	19
1.293.O.	30
1.191.O.	3
1.007.O.	4
1.042.O.	9
1.181.O.	3
1.086.O.	3
2.117.O.	29
1.232.O.	22
1.205.O.	28
1.285.O.	6
1.283.C.	6
1.261.O.	26
2.130.O.	11
1.215.A.	31
3.058.O.	34
1.063.O.	16
1.193.O.	16
1.187.O.	16
1.170.O.	16
1.161.O.	16
1.084.O.	20
1.121.O.	16
1.031.O.	13
1.255.O.	13
2.275.O.	13


In [14]:
from collections import defaultdict
pbonly=open("./data/asm1_blast_comparison_summed_matches.txt").readlines()

pbodict=defaultdict(lambda:set())

for l in pbonly[1:]:
    vec=l.split("\t")
    ctg=vec[0]
    phage=vec[1]
    pbodict[phage].add(ctg)

defaultdict(<function __main__.<lambda>>,
            {'1.007.O.': {'ctg7180000000098'},
             '1.028.O.': {'ctg7180000000156'},
             '1.029.O.': {'ctg7180000000158'},
             '1.030.O.': {'ctg7180000000154'},
             '1.042.O.': {'ctg7180000000109'},
             '1.052.A.': {'ctg7180000000159'},
             '1.054.O.': {'ctg7180000000121'},
             '1.063.O.': {'ctg7180000000088'},
             '1.074.O.': {'ctg7180000000127', 'ctg7180000000142'},
             '1.084.O.': {'ctg7180000000091'},
             '1.086.O.': {'ctg7180000000141'},
             '1.087.A.': {'ctg7180000000150'},
             '1.090.B.': {'ctg7180000000126'},
             '1.110.O.': {'ctg7180000000118'},
             '1.121.O.': {'ctg7180000000086'},
             '1.122.B.': {'ctg7180000000117'},
             '1.123.O.': {'ctg7180000000107'},
             '1.131.O.': {'ctg7180000000120'},
             '1.135.O.': {'ctg7180000000108'},
             '1.144.O.': {'ctg7180000000111'}

In [ ]:
from collections import defaultdict
import os
from pyfaidx import Fasta

if os.path.exists("./contigs/")==False:
    os.mkdir("./contigs/")

phages=open("K2.151228_PacBio_Nahant_Phages.txt").readlines()
phages=[i.replace("\r\n","") for i in phages]

pbonly=open("asm1_blast_comparison_summed_matches.txt").readlines()

pbodict=defaultdict(lambda:set())

for l in pbonly[1:]:
    vec=l.split("\t")
    ctg=vec[0]
    phage=vec[1]
    pbodict[phage].add(ctg)
    
kkhgap=open("kkhgap_blast_comparison_summed_matches.txt").readlines()
kkdict=defaultdict(lambda:set())

for k in kkhgap[1:]:
    vec=k.split("\t")
    ctg=vec[0]
    phage=vec[1]
    ct=float(vec[-1].replace("\n",""))
    if ct<0.15:
        kkdict[phage].add(ctg)
        
kkhgap=open("kkhgap2_blast_comparison_summed_matches.txt").readlines()
kkdict2=defaultdict(lambda:set())

for k in kkhgap[1:]:
    vec=k.split("\t")
    ctg=vec[0]
    phage=vec[1]
    ct=float(vec[-1].replace("\n",""))
    if ct<1:
        kkdict2[phage].add(ctg)

pbonlys=[("./nvp_pba_1/9-terminator/asm.ctg.fasta","pbcr1", pbodict), 
         ("/nobackup1/jbrown/nahant_phage_pacbio/SMRTportal_HGAPv2_defaults/kkHGAP.fasta","kkhgap1",kkdict),
        ("/nobackup1/jbrown/nahant_phage_pacbio/SMRTportal_KKHGAP_run2/kkhgap_run2.fasta", "kkhgap2", kkdict2)]
        
for p in phages:
    try1="./try1/nvp_%s_binned_asm_try1/" % p
    try2="./try2/nvp_%s_binned_asm_try2/" % p
    
    if os.path.exists(try1) or os.path.exists(try2) or p in pbodict.keys():
        print "some %s assemblies exist" % p
        outfasta=open("./contigs/%s/%sasm.tries.fasta" % (p,p), "w")
        hybrids=["%s9-terminator/asm.ctg.fasta" % try1, "%s9-terminator/asm.ctg.fasta" % try2]
        if os.path.exists("./contigs/%s" % p)==False:
            os.mkdir("./contigs/%s" % p)
        
        for h in hybrids:
            if os.path.exists(h):
                print "%s worked" % h
                f1=Fasta(h)
                for f in f1.keys():
                    single=open("./contigs/%s/%s%s.hybrid.asm.fasta" % (p, p, h.split("/")[1]),"w")
                    single.write(">%s%s\n%s\n" % (p, h.split("/")[1], str(f1[f])))
                    outfasta.write(">%s%s\n%s\n" % (p, h.split("/")[1], str(f1[f])))
                    single.close()         
        for assembly, name, cdict in pbonlys:
            f2=Fasta(assembly)
            ctgs=cdict[p]
            if len(ctgs)>0:
                print "%s has a match in the original %s assembly" % (p, name)
                for c in ctgs:
                    single=open("./contigs/%s/%s_%s.fasta" % (p, p, name),"w")
                    single.write(">%s_%s\n%s\n" % (c, name,str(f2[c])))
                    outfasta.write(">%s_%s\n%s\n" % (c, name,str(f2[c])))
                    single.close()
        outfasta.close()

>### Ran above on server using ipython

# Next

## draw gepard dot plots of illumina genomes versus kkhgap assemblies

## Then draw gepard dot plots of the hybrid assemblies

>### on engaging run in ipython:


In [ ]:
from collections import defaultdict
import os
import glob
import subprocess

os.chdir("/nobackup1/jbrown/programs/gepard-1.30/")

if os.path.exists("./comparisons/")==False:
    os.mkdir("./comparisons/")
    os.mkdir("./comparisons/gepard/")

phages=[i.replace("\r\n","") for i in open("K2.151228_PacBio_Nahant_Phages.txt").readlines()]

for phage in phages:
    illumina_asm="/nobackup1/jbrown/annotation/genomes/%sfinal.fasta" % phage
    if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%s_kkhgap1.fasta" % (phage, phage)):
        kkhgap=glob.glob("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/*kkhgap1.fasta" % phage)[0]
    
        output="/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/%s_illumina_vs_kkhgap.png" % phage
        gepard_cmd="bash /nobackup1/jbrown/programs/gepard-1.30/gepardcmd.sh -seq1 %s -seq2 %s -matrix /nobackup1/jbrown/programs/gepard-1.30/matrices/edna.mat -outfile %s" % (illumina_asm, kkhgap, output)
        subprocess.call(gepard_cmd.split(" "))
        print "%s comparison done" % phage
    

In [ ]:
for phage in phages:
    illumina_asm="/nobackup1/jbrown/annotation/genomes/%sfinal.fasta" % phage
    if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%stry1.hybrid.asm.fasta" % (phage, phage)):
        kkhgap=glob.glob("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%stry1.hybrid.asm.fasta" % (phage,phage))[0]
    
        output="/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/%s_illumina_vs_try1hybrid.png" % phage
        gepard_cmd="bash /nobackup1/jbrown/programs/gepard-1.30/gepardcmd.sh -seq1 %s -seq2 %s -matrix /nobackup1/jbrown/programs/gepard-1.30/matrices/edna.mat -outfile %s" % (illumina_asm, kkhgap, output)
        subprocess.call(gepard_cmd.split(" "))
        print "%s comparison done" % phage

In [ ]:
for phage in phages:
    illumina_asm="/nobackup1/jbrown/annotation/genomes/%sfinal.fasta" % phage
    if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%s_kkhgap2.fasta" % (phage, phage)):
        kkhgap=glob.glob("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/*kkhgap2.fasta" % phage)[0]
    
        output="/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/%s_illumina_vs_kkhgap2.png" % phage
        gepard_cmd="bash /nobackup1/jbrown/programs/gepard-1.30/gepardcmd.sh -seq1 %s -seq2 %s -matrix /nobackup1/jbrown/programs/gepard-1.30/matrices/edna.mat -outfile %s" % (illumina_asm, kkhgap, output)
        subprocess.call(gepard_cmd.split(" "))
        print "%s comparison done" % phage

In [ ]:
def run_gepard(fasta1, fasta2, output):
    os.chdir("/nobackup1/jbrown/programs/gepard-1.30/")
    gepard_cmd="bash /nobackup1/jbrown/programs/gepard-1.30/gepardcmd.sh -seq1 %s -seq2 %s -matrix /nobackup1/jbrown/programs/gepard-1.30/matrices/edna.mat -outfile %s" % (fasta1, fasta2, output)
    subprocess.call(gepard_cmd.split(" "))

In [ ]:
phage="2.130.O."
fasta1="/nobackup1/jbrown/annotation/genomes/%sfinal.fasta" % phage
fasta2="/nobackup1/jbrown/nahant_phage_pacbio/contigs/hgap2_to_circularize/2.130.O.hgap2.circularized.fasta"

run_gepard(fasta1, fasta2, "/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/2.130.O.illumina.vs.hgapcirc.png")

## Identification of cut sites for pacbio/hgap contigs that were larger than their illumina counterparts

> I want to cut the pacbio/hgap contigs at the start location of the illumina contigs

In [1]:
blast=open("./data/kkhgap2_vs_igenomes.txt").readlines()
print len(blast)

goodblast=[]

for l in blast:
    vec=l.split("\t")
    if float(vec[2])>99 and int(vec[3])>1000:
        goodblast.append(l)
print len(goodblast)

6731
149


In [2]:
from pyfaidx import Fasta
#create text object containing information about pb assembly contig lengths:

kkasm=Fasta("./data/kkhgap_run2.fasta") 

kk_lens={}

for s in kkasm.keys():
    length=len(str(kkasm[s]))
    kk_lens[s]=length
    
ilens = open("./data/igenome_lengths.txt").readlines()
phages = [i.split("\t")[0] for i in ilens[1:]]
lens = [i.split("\t")[1].replace("\n","") for i in ilens[1:]]
i_lens = dict(zip(phages, lens))

In [8]:
from collections import defaultdict
starts = ""
starts = defaultdict(lambda: (200000,200000))

#identifying earliest point on target contig where query hits, the corresponding match on the query contig is where we will cut it for circularization
for l in goodblast:
    vec = l.split("\t")
    q = vec[0]
    t = vec[1].split("_")[1]
    if int(kk_lens[q]) > int(i_lens[t]):
        qstart = vec[6]
        tstart = vec[8]
        if int(tstart) < starts[t][1]:
            #print "changing %s to start %s instead of %s" % (t, tstart, starts[q][1])
            starts[t] = (int(qstart), int(tstart))

for l in goodblast:
    vec = l.split("\t")
    q = vec[0]
    t = vec[1].split("_")[1]
    qstart = vec[6]
    qstop = vec[7]
    tstart = vec[8]
    tstop = vec[9]
    if int(kk_lens[q]) > int(i_lens[t]) and abs(int(kk_lens[q])-int(i_lens[t])) > 200:
        if starts[t][1] == 1:
            #print "a\t"+str(starts[t])
            continue
        elif starts[t][0] == 1:
            #print "%s\t%s\t%s\t%s\t" % (qstart, qstop, tstart, tstop)
            starts[t] = (int(qstop),int(tstop))
            #print starts[t]

out = open("./data/kkhgap2_cut_spots.txt","w")
for s in starts.keys():
    out.write("%s\t%s\n" % (s, starts[s][0]))
out.close()

In [9]:
!head ./data/kkhgap2_cut_spots.txt
!scp ./data/kkhgap2_cut_spots.txt jbrown@eofe4.mit.edu:/nobackup1/jbrown/nahant_phage_pacbio/kkhgap2_cut_spots.txt

2.096.O.	109
1.211.B.	47116
1.257.O.	29093
2.117.O.	63692
1.161.O.	82616
1.232.O.	36421
1.054.O.	27426
1.190.O.	11155
2.130.O.	55260
1.193.O.	85049


### Create split contig files for recircularization
> ### run in ipython on engaging

In [ ]:
from pyfaidx import Fasta
import os

'''
splitting the contig in order to submit it for circularization
if the split point is too early or late in the contig, the contig won't circularize
so I have to put the split point in the middle of the contig by doing a little rearranging
'''

if os.path.exists("./contigs/hgap2_to_circularize/")==False:
    os.mkdir("./contigs/hgap2_to_circularize/")

spots=open("kkhgap2_cut_spots.txt").readlines()
starts=dict(zip([i.split("\t")[0] for i in spots], [int(i.split("\t")[1].replace("\n","")) for i in spots]))

for phage in starts.keys():
    if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%s_kkhgap2.fasta" % (phage, phage)):
        contig=Fasta("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%s_kkhgap2.fasta" % (phage, phage))
        l=len(str(contig[0]))
        spot=starts[phage]
        if spot < 10000:
            split1 = str(contig[0][l/2:l])+str(contig[0][0:spot])
            split2 = str(contig[0][spot:l/2])
        elif spot>l-10000:
            split1=str(contig[0][l/2:spot])
            split2=str(contig[0][spot:l])+str(contig[0][0:l/2])
        else:
            split1=str(contig[0][0:spot])
            split2=str(contig[0][spot:])
        out=open("./contigs/hgap2_to_circularize/%shgap.to_circularize.fasta" % phage, "w")
        out.write(">%s\n%s\n>%s_1\n%s" % (contig[0].name,split1,contig[0],split2))
        out.close()


# Circularizing hgap2 contigs with terminal overlap

In [ ]:
# %load ./scripts/recircularize.py
#!usr/bin/python

import glob
import subprocess
import os

def run_gepard(fasta1, fasta2, output):
    os.chdir("/nobackup1/jbrown/programs/gepard-1.30/")
    gepard_cmd="bash /nobackup1/jbrown/programs/gepard-1.30/gepardcmd.sh -seq1 %s -seq2 %s -matrix /nobackup1/jbrown/programs/gepard-1.30/matrices/edna.mat -outfile %s" % (fasta1, fasta2, output)
    subprocess.call(gepard_cmd.split(" "))

to_cir=glob.glob("/nobackup1/jbrown/nahant_phage_pacbio/contigs/hgap2_to_circularize/*hgap.to_circularize.fasta")
phages=[i.split("/")[-1].split("hgap")[0] for i in to_cir]

cp=zip(to_cir, phages)

if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/hgap2_circularized/")==False:
    os.mkdir("/nobackup1/jbrown/nahant_phage_pacbio/contigs/hgap2_circularized/")

if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/recirc/")==False:
    os.mkdir("/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/recirc/")


for split_fasta, phage in cp:
    output_afg="/nobackup1/jbrown/nahant_phage_pacbio/contigs/hgap2_circularized/%shgap2.cir.afg" % phage
    amos_cmd="toAmos -s %s -o %s" % (split_fasta, output_afg)
    minimus_cmd="minimus2 %s" % output_afg.replace(".afg","")

    subprocess.call(amos_cmd.split(" "))
    subprocess.call(minimus_cmd.split(" "))

    illumina_asm="/nobackup1/jbrown/annotation/genomes/%sfinal.fasta" % phage
    cir_asm=output_afg.replace(".afg",".fasta")
    gpd_out="/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/recirc/%sillumina_vs_hgap2circ.png" % phage
    run_gepard(illumina_asm, cir_asm, gpd_out)


# pool genomes back together, compare to original icontigs

In [ ]:
from Bio import SeqIO
import os

'''
outputs an assessment of whether or not circularization worked, and transfers all contigs, 
circularized or not to new fasta file.
'''

path="/nobackup1/jbrown/nahant_phage_pacbio"
outfasta=open("%s/contigs/hgap2_processed_contigs.fasta" % path,"w")

phages=open("K2.151228_PacBio_Nahant_Phages.txt").readlines()
phages=[i.replace("\r\n","") for i in phages]
seq=range(0,len(phages))

ps=zip(phages,seq)

out=open("pb_results_circ.txt","w")

for p,s in ps:
    cir_fasta="/nobackup1/jbrown/nahant_phage_pacbio/contigs/hgap2_circularized/%shgap2.cir.fasta" % p
    hgap2_fasta="%s/contigs/%s/%s_kkhgap2.fasta" % (path, p, p)
    
    if os.path.exists(cir_fasta) and os.path.getsize(cir_fasta)>0:
        print "%s circular" % p
        fasta=cir_fasta
        out.write("%s\tcircularized\n" % p)
    elif os.path.exists(hgap2_fasta):
        print "%s original" % p
        out.write("%s\tnon-circular\n")
        fasta=hgap2_fasta
    else:
        print "%s did not have an hgap2 assembly" % p
        out.write("%s\tno_hgap_contigs\n")
        continue
    
    f=open(fasta)

    record = SeqIO.read(f, "fasta")
    outfasta.write(">%s_kkhgap2_processed\n%s\n" % (p,record.seq))

out.close()    
outfasta.close()       

### output:
```
1.080.O. did not have an hgap2 assembly
1.043.O. did not have an hgap2 assembly
1.095.O. did not have an hgap2 assembly
1.107.C. did not have an hgap2 assembly
1.020.O. did not have an hgap2 assembly
1.008.O. did not have an hgap2 assembly
1.249.A. did not have an hgap2 assembly
1.190.O. circular
1.259.O. circular
2.159.B. did not have an hgap2 assembly
1.028.O. did not have an hgap2 assembly
1.257.O. circular
1.082.O. circular
1.176.O. circular
1.228.O. circular
1.148.O. circular
1.233.B. circular
1.189.O. circular
1.182.O. original
1.074.O. circular
1.165.O. did not have an hgap2 assembly
1.154.O. original
1.211.B. circular
1.110.O. original
1.275.O. original
1.131.O. original
1.239.O. circular
1.111.B. did not have an hgap2 assembly
1.216.O. circular
12G01.O. did not have an hgap2 assembly
1.054.O. circular
1.090.B. circular
1.135.O. circular
1.236.O. did not have an hgap2 assembly
1.052.A. original
1.209.O. original
1.185.O. circular
1.291.O. circular
1.077.O. did not have an hgap2 assembly
1.204.O. did not have an hgap2 assembly
1.151.O. original
1.276.O. original
1.144.O. original
1.009.O. did not have an hgap2 assembly
1.122.B. original
1.231.O. original
2.095.B. circular
2.096.O. original
2.044.O. original
1.177.O. original
1.087.A. original
1.030.O. original
1.123.O. circular
1.029.O. circular
1.253.O. original
1.262.O. original
1.210.O. circular
1.293.O. original
1.191.O. original
1.007.O. circular
1.042.O. circular
1.181.O. did not have an hgap2 assembly
1.086.O. original
2.117.O. circular
1.232.O. circular
1.205.O. circular
1.285.O. original
1.283.C. circular
1.261.O. circular
2.130.O. circular
1.215.A. circular
3.058.O. original
1.063.O. circular
1.193.O. circular
1.187.O. circular
1.170.O. circular
1.161.O. circular
1.084.O. original
1.121.O. circular
1.031.O. did not have an hgap2 assembly
1.255.O. circular
2.275.O. circular
```

In [ ]:
'''
was circularization successful?  
I noticed that some of the circularized ouput from hgap2 were blank, 
suggesting that the circularization step failed.
going to check those files to see which ones had a contig present, and which ones did not.
'''

phages=open("K2.151228_PacBio_Nahant_Phages.txt").readlines()
phages=[i.replace("\r\n","") for i in phages]

for p in phages:

    cir_fasta="/nobackup1/jbrown/nahant_phage_pacbio/contigs/hgap2_circularized/%shgap2.cir.fasta" % p
    
    if os.path.exists(cir_fasta) and os.path.getsize(cir_fasta)==0:
        print "%s failed circularization" % p
        
    elif os.path.exists(cir_fasta) and os.path.getsize(cir_fasta)>0:
        print "%s circularized" % p
        
    else:
        print "%s not circularized" % p

### output:
```
1.080.O. not circularized
1.043.O. not circularized
1.095.O. not circularized
1.107.C. not circularized
1.020.O. not circularized
1.008.O. not circularized
1.249.A. not circularized
1.190.O. circularized
1.259.O. circularized
2.159.B. not circularized
1.028.O. not circularized
1.257.O. circularized
1.082.O. circularized
1.176.O. circularized
1.228.O. circularized
1.148.O. circularized
1.233.B. circularized
1.189.O. circularized
1.182.O. failed circularization
1.074.O. circularized
1.165.O. not circularized
1.154.O. failed circularization
1.211.B. circularized
1.110.O. failed circularization
1.275.O. failed circularization
1.131.O. failed circularization
1.239.O. circularized
1.111.B. not circularized
1.216.O. circularized
12G01.O. not circularized
1.054.O. circularized
1.090.B. circularized
1.135.O. circularized
1.236.O. not circularized
1.052.A. not circularized
1.209.O. not circularized
1.185.O. circularized
1.291.O. circularized
1.077.O. not circularized
1.204.O. not circularized
1.151.O. failed circularization
1.276.O. failed circularization
1.144.O. failed circularization
1.009.O. not circularized
1.122.B. failed circularization
1.231.O. failed circularization
2.095.B. circularized
2.096.O. failed circularization
2.044.O. not circularized
1.177.O. failed circularization
1.087.A. not circularized
1.030.O. not circularized
1.123.O. circularized
1.029.O. circularized
1.253.O. not circularized
1.262.O. failed circularization
1.210.O. circularized
1.293.O. failed circularization
1.191.O. not circularized
1.007.O. circularized
1.042.O. circularized
1.181.O. not circularized
1.086.O. not circularized
2.117.O. circularized
1.232.O. circularized
1.205.O. circularized
1.285.O. not circularized
1.283.C. circularized
1.261.O. circularized
2.130.O. circularized
1.215.A. circularized
3.058.O. not circularized
1.063.O. circularized
1.193.O. circularized
1.187.O. circularized
1.170.O. circularized
1.161.O. circularized
1.084.O. failed circularization
1.121.O. circularized
1.031.O. not circularized
1.255.O. circularized
2.275.O. circularized
```